In [1]:
import sys
sys.path.append('../')

In [2]:
from application.name_obj_classes import PubName, PersonName, remove_punct

In [3]:
from application.review_obj_class import ReviewObj

In [4]:
from application.text_preprocessing import preprocess_text

In [5]:
import os
import pandas as pd
import re
from collections import Counter
import numpy as np
import pickle
from nltk.metrics import edit_distance
%pprint

Pretty printing has been turned OFF


In [6]:
# loading in files
directory = "../../aps_reviews_50/aps_reviews/"
filenames = os.listdir(directory)
txts = []
for file in filenames:
    with open(directory + file) as f:
        txts.append(f.read())

In [7]:
[x.split('.')[0] for x in filenames]

['136726613', '90142613', '90390665', '124486140', '126315326', '136992742', '137651039', '89668273', '136971619', '90313218', '128376070', '124688361', '124882533', '89639059', '88461015', '124190789', '136902134', '90180084', '137244878', '88721263', '137412078', '89675095', '760950213', '124217248', '853770606', '137195890', '125727674', '88797650', '89649761', '90967694', '89672866', '89674750', '89635917', '124196531', '128176513', '88477095', '125715359', '124189638', '136887509', '90554411', '90186165', '125502663', '125724646', '124740955', '124707428', '89644579', '137072896', '136685639', '126788760', '89671107']

## Part 1: Publisher Names

In [8]:
def clean_pub_matches(match_list):
    cleaned_matches = []
    for match in match_list:
        index_to_start = 0
        for i, x in enumerate(match[1].split()):
            if x[0].islower() and x[0]!='&':
                index_to_start = i+1
        cleaned_matches.append(' '.join(match[1].split()[index_to_start:]))
    return cleaned_matches

In [9]:
def remove_dash_for_pub(pub_match):
    return re.sub(r'(?<!/w)-(?!/w)', '', pub_match)

In [10]:
pub_ends = ['company','co','incorporated','inc','firm','press','group','publishers','publishing',
                    'publications','pub','books','ltd','limited','society','house','associates']

In [11]:
pub_ends_list = '|'.join([x.capitalize()+'\.?(?!\w)' for x in pub_ends])

In [12]:
pub_ends.extend([x.capitalize() for x in pub_ends])

In [13]:
pub_ends

['company', 'co', 'incorporated', 'inc', 'firm', 'press', 'group', 'publishers', 'publishing', 'publications', 'pub', 'books', 'ltd', 'limited', 'society', 'house', 'associates', 'Company', 'Co', 'Incorporated', 'Inc', 'Firm', 'Press', 'Group', 'Publishers', 'Publishing', 'Publications', 'Pub', 'Books', 'Ltd', 'Limited', 'Society', 'House', 'Associates']

``` 
    for e, index in enumerate(indices):
        
        if (e==len(indices)-1):
            end_index = -1
        else:
            end_index = indices[e+1][0]
        
        end_span = len(txt[indices[e][0]:end_index])   
        get_match = re.finditer('[A-Z]\w+[^A-Z]|[A-Z].[^A-Z]', txt[indices[e][0]:end_index])
        matches = [(m.span(), m.group()) for m in get_match]
        matches.reverse()
        
        for n, m in enumerate(matches):
            if n<len(matches)-1:
                if (m[0][1] != matches[n-1][0][0]):
                    end_span = m[0][1]
        
        result = txt[indices[e][0]:(indices[e][0] + end_span - 1)]
        
        if len(result) > len(indices[e][1]):
            names.append(txt[indices[e][0]:(indices[e][0] + end_span - 1)])
            spans.append(indices[e][0])
```

+ split into tokens
+ stop as soon 
+ stop as city names
+ put word tokenizer in review obj

In [14]:
review_list = []
for filename, txt in zip([x.split('.')[0] for x in filenames], txts):
    review_list.append(ReviewObj(filename, txt))

In [15]:
toks = review_list[0].cleaned_toks

In [16]:
toks

['LITERARY', 'NOTES', '.', '-The', 'Chautauqua', 'department', 'of', '``', 'Wide', 'Awake', '``', 'is', 'now', 'published', 'separately', '.', '-Mr.', 'Charles', 'Dudley', 'Warner', 'has', 'written', 'for', '``', 'The', 'North', 'American', 'Review', '``', 'a', 'paper', 'on', 'prison', '.', '-Mr.', 'Vedder', "'s", 'illustrations', 'of', 'the', '``', 'Rubfilyfit', '``', 'have', 'found', 'a', 'great', 'admirer', 'in', 'the', 'Queen', 'of', 'Italy', '.', '-Cassell', '&', 'Co.', 'sill', 'shortly', 'publish', 'a', '``', 'Dictionary', 'of', 'English', ',', '``', 'by', 'Sidney', 'J', '.', 'Low', 'and', 'F.', 'S.', 'Pulling', '.', '-Professor', 'Thorold', 'Rogers', 'is', 'writing', 'a', 'worL', 'on', 'the', '``', 'of', 'the', 'Privileges', 'of', 'British', 'Citizenship.', '``', '-', '``', 'Mollie', 'Carew', '``', 'is', 'the', 'title', 'of', 'a', 'pleasant', 'story', 'by', 'A.', 'M.', 'W.', ',', 'just', 'published', 'by', 'E.', 'P.', 'Dutton', '&', 'Co.', ',', 'New', 'Yor', "'", '.', '-', '.-rs

In [77]:
def is_part_of_pub(pub_part):
    if (pub_part == 'and') or (pub_part =='&'):
        return True
    else:
        return pub_part[0].isupper()

In [78]:
is_part_of_pub('Egg')

True

In [103]:
pubnames = []
for e, tok in enumerate(toks):
    if tok.replace(",","").replace(".","").replace('-',"") in pub_ends:
        if is_part_of_pub(toks[e-1]):
            pub_name = [tok]
            pub_span = []
            for pos in range(e-1, e-6, -1):
                if toks[pos] == '.':
                    break
                elif not is_part_of_pub(toks[pos]):
                    break
                pub_name.append(toks[pos])
                pub_span.append(pos)
            pubnames.append((e, pub_span[-1]))

In [104]:
pubnames

[(56, 54), (117, 113), (150, 149), (410, 406), (498, 496), (706, 702), (777, 775), (913, 912), (973, 972)]

In [73]:
indices

[56, 118, 409, 497, 588, 706]

In [60]:
def get_publishers(review):
    """
    Takes a ReviewObj. 
    Returns a list of potential publishers. Searches using pub_ends, capitalization, and associates.
    
    For reference:
    -------------
    pub_ends = ['co','company','inc','incorporated','firm','press','group', 'pub','publishers','publishing',
                    'publications','books','ltd','limited','society','house','associates']
                    
    pub_associates = ['sons','son','brother','brothers']
    
    """
    
    pubs = []
    spans = []
    
    txt = review.cleaned_text

    p_iter = re.finditer(pub_ends_list, txt)
    p_indices = [(m.end(), m.group()) for m in p_iter]
    
    for e, index in enumerate(p_indices):
        if (e==0):
            start_index = 0
        else:
            start_index = p_indices[e-1][0] - 1
            
        start_span = 0
        
        match = re.finditer("[A-Z]\w+| & | and ", txt[start_index:index[0]])
#         match = re.finditer("(?<= [^A-Z&\.])[\S]{,10} ?[A-Z][\w&. ]*?" + index[1] + '(?!\w)', 
#                                 review.cleaned_text[start_index:index[0]])
        #all_matches = [(m.span(), remove_dash_for_pub(m.group())) for m in match]
        #print(all_matches)
        matches = [(m.span(), m.group()) for m in match]
        
        for n, m in enumerate(matches):
            #print(n,m)
            if n<len(matches)-1:
                #print(m[0][1], matches[n+1][0][0])
                if (m[0][1] != matches[n+1][0][0]):
                    start_span = matches[n+1][0][0]
                    #print(start_span)
        
        result = txt[start_index + start_span:index[0]]
        #print(result)
        
        if len(result) > len(p_indices[e][1]):
            pubs.append(result)
            spans.append(p_indices[e][0])


    pubs = [PubName(word) for word in pubs]
    
    for pub in pubs:
        pub.review_id = review.review_id
    
    for e, pub in enumerate(pubs):
        pub.review_id = review.review_id
        pub.review_loc = spans[e]
    
    return pubs

In [61]:
for filename, txt in zip([x.split('.')[0] for x in filenames],txts):
    rev = ReviewObj(filename, txt)
    print(get_publishers(rev))

['Cassell & Co.', 'Dutton & Co.', 'Barnes & Co.', 'Heath & Co.', 'Crowell & Co.']
['Holt & Co.']
[]
[]
[]
[]
['GINN & Co.', 'GINN & Co.', 'KnRR & Co.', 'Gl1N & Co.', 'Ginn & Co.', 'LIPizxcoTr & Co.', 'BROWN & Co.', ' & Co.', ' & Co.', ' & Co.', 'BARNES & Co.', 'LpnIrrcorr & Co.', 'RANDoLPHn & Co.']
['Mead & Co.']
[]
[' & Co.']
[]
[]
[]
[]
[]
['Brown & Co.', 'Slead & Co.']
['Sou & Co.']
['Green & Co.', 'Mead & Co.', 'Randolph & Co.', 'Macmillan & Co.', 'Mifflin & Co.']
[]
[]
[]
[]
[]
[' & Co.', 'Macmillan & Co.']
[]
[]
[]
['Ginn and Company', 'g Company', 'g Company']
['Macmillan & Co.']
[]
[]
['Mead & Co.', 'Dutton & Co.', 'Maynard & Co.']
['Macmillan & Co.']
[]
[]
['o Publishers']
['MerriUCompany.']
[]
[]
['Dutton & Co.']
[]
['CroweU & Co.']
[]
[]
[]
[]
['Senate and House']
['Appleton & Co.']
[]
['AtiSlin & Co.']


In [18]:
for filename, txt in zip([x.split('.')[0] for x in filenames],txts):
    rev = ReviewObj(filename, txt)
    print(filename)
    if rev.person_names:
        for x in rev.pub_names:
            print(x)
        print()

136726613
E. P. Dutton & Co.
A. S. Barnes & Co.
Ginn, Heath & Co.
E. P. Dutton & Cov
Thomas Y. Crowell & Co.
American Historical Society
American Historical Society
Dunlap Society

90142613
The Gadfly. By E. L. VWynich. Henry Holt & Co.

90390665

124486140

126315326
136992742
137651039
BOSTON: GINN & Co.
CAGO: CHAS. 1I. KnRR & Co.
American Geographical Society
BosTON: Gl1N & Co.
Messrs. Ginn & Co.
A. S. BARNES & Co.
BOSTON AND LONDON GINN & Co.
BOSTON: GINN & Co.
CAGO: CHAS. 1I. KnRR & Co.
BosTON: Gl1N & Co.
Messrs. Ginn & Co.
A. S. BARNES & Co.

89668273

136971619
90313218

128376070

124688361
124882533

89639059
Publishers

88461015
Robert Clarke Co
Graphic Company

124190789
259. J. B. Lippincott Co.
05. Macmillan Co.
259. J. B. Lippincott Co.
05. Macmillan Co.
404. Maemillall Co.

136902134

90180084
Mead & Co.
& Co.
Mead & Co.
& Co.
Houghton, Mifflin & Co.

137244878

88721263
137412078

89675095

760950213
124217248
J. B. Lippincott Co.
American Book Co.
Macmillan & Co.
Arena

## Part 1: Person Names

In [12]:
titles = """Doctor,Dr,Mr,Mrs,Miss,Msgr,Monsignor,Rev,Reverend,Hon,Honorable,Honourable,Prof,Professor,Madame,Madam,Lady,Lord,Sir,Dame,Master,Mistress,Princess,Prince,Duke,Duchess,Baron,Father,Chancellor,Principal,President,Pres,Warden,Dean,Regent,Rector,Provost,Director
"""

In [13]:
titles = titles.rstrip().split(',')

In [14]:
title_list = '\.?\s(?=[A-Z])|'.join(titles)

In [15]:
def remove_punct_not_following_title_or_initial(name):
    name_parts = name.split()
    cleaned_name = []
    for part in name_parts:
        if part[-1] in '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~':
            if (len(part)>2) and (part[:-1] not in titles):
                cleaned_name.append(part[:-1])
            else:
                cleaned_name.append(part)
        else:
            cleaned_name.append(part)
    return ' '.join(cleaned_name)

In [16]:
def clean_name(name):
    name = remove_punct_not_following_title_or_initial(name)
    cleaned_name = []
    return ' '.join([word for word in name.split() if (word[0].isalpha())])

In [17]:
def getCapitalizedWords(txt):
    """
    Returns strings of capitalized words up 3 words long. 
    Removes words/phrases containing stopwords and words found later in the text lowercased.
    """
    #listen idk why it won't just let me put in an optional repeat either
    all_words = []
    
    #three words
    all_words.extend([match for match in re.findall('[A-Z]\S* [A-Z]\S* [A-Z]\S+', txt) if 
                      all([(remove_punct(word).lower() not in stopword_list) for word in match.split()]) 
                      and all([(remove_punct(word).lower() not in txt) for word in match.split()])])
    
    #two words
    two_words = [match for match in re.findall('[A-Z]\S* [A-Z]\S+', txt) if 
                      all([(remove_punct(word).lower() not in stopword_list) for word in match.split()]) 
                      and all([(remove_punct(word).lower() not in txt) for word in match.split()])
                      and all([match not in x for x in all_words])]
    
    all_words.extend(two_words)
    
    #one word
    one_words = [match for match in re.findall('[A-Z]\S+', txt) if 
                      (remove_punct(match).lower() not in stopword_list) 
                      and (remove_punct(match).lower() not in txt)
                      and all([match not in x for x in all_words])]
    
    all_words.extend(one_words)
    
    return [word for word in [' '.join([removePunct(y) for y in x.split() if remove_punct(y) not in titles]) 
                      for x in all_words] if (len(word)>1)]

In [18]:
def consolidateNames(name_list):
    """
    Takes list of AuthNames and returns list of lists, consolidating by likely identical authors.
    """
    name_set = []
    used_indices = []
    last_names = sorted([name.last_name for name in name_list], key=len)
    
    for i, name in enumerate(last_names):
        if i not in used_indices:
            full_name = [x for x in name_list if x.last_name == name][0]
            name_holder = [full_name]
            for j, name2 in enumerate(last_names):
                full_name2 = [x for x in name_list if x.last_name == name2][0]
                if (i < (len(last_names) - 1)) and (i!=j):
                    if (edit_distance(name, name2[:len(name)+1]) < 2) and (j not in used_indices):
                        if (full_name.first_initial==full_name2.first_initial and full_name.middle_initial==full_name2.middle_initial or full_name.title==full_name2.title) or (full_name.first_initial==full_name2.first_initial or full_name.middle_initial==full_name2.middle_initial and full_name.title==full_name2.title):
                            name_holder.append(full_name2)
                            used_indices.append(j)
            used_indices.append(i)
            name_set.append(name_holder)
    
    return name_set                  

In [19]:
def get_names_following_titles(review):
    """
    Returns names following titles - specifically capitalized titles followed by capitalized names.
    Names can be any number of words in length, and can include punctuation.
    
    """
    names = []
    spans = []
    
    txt = review.cleaned_text

    iterx = re.finditer(title_list, txt)
    indices = [(m.start(), m.group()) for m in iterx]
    
    for e, index in enumerate(indices):
        
        if (e==len(indices)-1):
            end_index = -1
        else:
            end_index = indices[e+1][0]
        
        end_span = len(txt[indices[e][0]:end_index])   
        get_match = re.finditer('[A-Z]\w+[^A-Z]|[A-Z].[^A-Z]', txt[indices[e][0]:end_index])
        matches = [(m.span(), m.group()) for m in get_match]
        matches.reverse()
        
        for n, m in enumerate(matches):
            if n<len(matches)-1:
                if (m[0][1] != matches[n-1][0][0]):
                    end_span = m[0][1]
        
        result = txt[indices[e][0]:(indices[e][0] + end_span - 1)]
        
        if len(result) > len(indices[e][1]):
            names.append(txt[indices[e][0]:(indices[e][0] + end_span - 1)])
            spans.append(indices[e][0])
        
    names = [word.replace("'s", "") for word in names]
    names = [PersonName(clean_name(word)) for word in names]
    
    for e, name in enumerate(names):
        name.review_id = review.review_id
        name.review_loc = (spans[e], spans[e]+len(name))
    
    return names

## ReviewObject

In [21]:
class ReviewObj():
    
    def __findnames(self):
        self.pub_names = get_publishers(self)
        self.person_names = get_names_following_titles(self)
        
    def __init__(self, aps_id, txt):
        self.review_id = aps_id
        self.original_text = txt
        self.cleaned_text = preprocess_text(txt)
        
        self.__findnames()

In [22]:
review_ex = ReviewObj(136726613, txts[0])

In [23]:
for x in review_ex.person_names:
    print(x)

Mr. Charles Dudley Warner
Mr. Vedder
Professor Thorold Rogers
Mrs. Browning
Mr. Swinburne
Dr. William
Dr. Geor6 Ebers
Mr. Julian Hawthorne
Mr. Julius Chambers
Mr. Cushing
Dr. Herbert B. Adam
President Andrew D. White
Mr. Brander Matthews
Dr. Joseph Parktercontinueshiseipositionuin
Professor E. Johnsou
Dr. Malaren
Dr. Parker


In [24]:
for filename, txt in zip([x.split('.')[0] for x in filenames],txts):
    rev = ReviewObj(filename, txt)
    print(filename)
    if rev.person_names:
        for x in rev.person_names:
            print(x)
        print()

136726613
Mr. Charles Dudley Warner
Mr. Vedder
Professor Thorold Rogers
Mrs. Browning
Mr. Swinburne
Dr. William
Dr. Geor6 Ebers
Mr. Julian Hawthorne
Mr. Julius Chambers
Mr. Cushing
Dr. Herbert B. Adam
President Andrew D. White
Mr. Brander Matthews
Dr. Joseph Parktercontinueshiseipositionuin
Professor E. Johnsou
Dr. Malaren
Dr. Parker

90142613
Mr. E. L. Voynich

90390665
Miss HowAiD
Mrs. A. L. Wister
Princess Mercedes
Miss Peard
Miss Peard
Lord Mttdhursm
Miss Peard
Sir Henry Lancaster

124486140
Dr. Atwood

126315326
136992742
137651039
Lord Clive
Mr. Alden
Mr. Bixby
Mr. Crane
Mr. Cook
Mr. Cook
Prof. Tillman
Mr. Thomas A. Edison
Mr. Tompkins
Mr. Haggard
Mr. Arthur Gillman
Mr. Gillman
Dr. Brown-Sequard
Dr. Brown-Sequard
Dr. Variot

89668273
Mr. Kenneth Grahame
Mr. Maxfield Parrish
Mr. Parrish
Mr. Swinburne
Mr. Arthur Symons
Mr. Havelock Ellis
Mr. A. Wilson Verity
Mr. George Saintsbury

136971619
90313218
Mr. Cabot Lodge
Mr. Adams
Mr. Morse
Mr. Oallatir
Mr. Htevens
Mr. Lodge
Mr. Gallatin

In [ ]:
for x in review_ex.person_names:
    print(x)

spacy NLP object steal their code

In [ ]:
for txt, filename in zip(txts, [x.split('.')[0] for x in filenames]):
    print(filename)
    docnames = get_names_following_titles(txt, filename)
    for name in docnames:
        print(name, name.review_loc)
    print()

## Goals
Make each Person name aware of others and able to check for potential matches?